# [Distributed statistical inference with `pyhf`](https://indico.cern.ch/event/1019958/contributions/4418598/)

## Cursorary introduction of `pyhf`

For the sake of brevity and time, we won't go into a full discussion of what `pyhf` is and what you can do with it. For now we'll point you to the [latest `pyhf` tutorial for `pyhf` `v0.6.2`](https://github.com/pyhf/pyhf-tutorial/tree/786702385e003511bbce27773c48df8769dfcfcb).

Very shortly though, `pyhf` is a pure-Python implimentation of the `HistFactory` family of statistical models

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyhf
from pyhf.contrib.viz import brazil

In [ ]:
pyhf.set_backend("numpy")
model = pyhf.simplemodels.uncorrelated_background(
    signal=[10.0], bkg=[50.0], bkg_uncertainty=[7.0]
)
data = [55.0] + model.config.auxdata

poi_vals = np.linspace(0, 5, 41)
results = [
    pyhf.infer.hypotest(
        test_poi, data, model, test_stat="qtilde", return_expected_set=True
    )
    for test_poi in poi_vals
]

fig, ax = plt.subplots()
fig.set_size_inches(7, 5)
plot = brazil.plot_results(poi_vals, results, ax=ax)

In [ ]:
from time import sleep

import funcx
from funcx.sdk.client import FuncXClient

## Introduction to `funcX`

## Demo of `funcX`

### Endpoint Creation

With the `funcx-endpoint` CLI API

In [ ]:
! funcx-endpoint --help

you need to create a template environment for your endpoint.

```
$ funcx-endpoint configure pyhf
```

Which will create a default `funcX` configuration file at `~/.funcx/pyhf/config.py`.

1. Note that `funcX` requires the use of [Gloubs](https://www.globus.org/) and so will require you to first login to a Globus account to use the `funcx-sdk`. Globus allows authentication through existing organizational logins or through Google accounts or [ORCID iD](https://orcid.org/) so this shouldn't be a barrier to use.
2. Once you authenticate with Globus you'll then need to approve the `funcx-sdk`'s required permissions and you'll be given a time limited authorization code.
3. Copy this code and paste it back into your terminal you ran `funcx-endpoint configure pyhf` in where you're asked to "Please Paste your Auth Code Below"

Upon success you'll see

```
A default profile has been create for <pyhf> at /home/jovyan/.funcx/pyhf/config.py
Configure this file and try restarting with:
    $ funcx-endpoint start pyhf
```

> If you're following along you'll want to switch over to a terminal to make this part easier

In [ ]:
! echo "funcx-endpoint configure pyhf"
! ls -l ~/.funcx/pyhf/config.py

In [ ]:
! cat ~/.funcx/pyhf/config.py

We'll go a step further though and use a prepared `funcX` configuration found under `funcX/binder-config.py`.

In [ ]:
! cp funcX/binder-config.py ~/.funcx/pyhf/config.py

and then start the endpoint

In [ ]:
! funcx-endpoint start pyhf

and you can verify that it is registered and up

In [ ]:
! funcx-endpoint list

**N.B.**: You'll want to take careful note of this `uuid` as this is the endpoint ID that you'll have your `funcX` code use.

A good way to deal with this is to save it in a `endpoint_id.txt` file that is ignored from version control.

In [ ]:
! funcx-endpoint list | grep pyhf | awk '{print $(NF-1)}' > endpoint_id.txt
! cat endpoint_id.txt

## Using funcX for (Fitting) Functions as a Service (FaaS)

### Prepare Functions

In [ ]:
def simple_example(backend="numpy", test_poi=1.0):
    import time

    import pyhf

    pyhf.set_backend(backend)

    tick = time.time()
    model = pyhf.simplemodels.uncorrelated_background(
        signal=[12.0, 11.0], bkg=[50.0, 52.0], bkg_uncertainty=[3.0, 7.0]
    )

    data = model.expected_data(model.config.suggested_init())
    return {
        "cls_obs": float(
            pyhf.infer.hypotest(test_poi, data, model, test_stat="qtilde")
        ),
        "fit-time": time.time() - tick,
    }

In [ ]:
simple_example()

In [ ]:
# Initialize funcX client
fxc = FuncXClient()
fxc.max_requests = 200

In [ ]:
with open("endpoint_id.txt") as endpoint_file:
    pyhf_endpoint = str(endpoint_file.read().rstrip())

In [ ]:
# register functions
infer_func = fxc.register_function(simple_example)

In [ ]:
# Run on endpoint
task_id = fxc.run(
    backend="numpy", test_poi=1.0, endpoint_id=pyhf_endpoint, function_id=infer_func
)

In [ ]:
# wait for it to run
sleep(5)

In [ ]:
# retrieve output
result = fxc.get_result(task_id)

In [ ]:
result

In [ ]:
# Run a different test POI
task_id = fxc.run(
    backend="numpy", test_poi=2.0, endpoint_id=pyhf_endpoint, function_id=infer_func
)
sleep(5)
result = fxc.get_result(task_id)

In [ ]:
result

## funcX endpoint shutdown

To stop a funcX endpoint from running simple use the `funcx-endpoint` CLI API again

In [ ]:
! funcx-endpoint stop pyhf
! funcx-endpoint list